In [1]:
from code_app.obtener_precedentes import obtener_precedentes_completos
from code_app.filtrar_raices import filtrar_valores_raiz
from code_app.simular_impacto_raices import simular_impacto_raices
from code_app.filtrar_impactos import filtrar_mayores_impactos
from tabulate import tabulate  # pip install tabulate

In [2]:
ruta_excel = "/Users/alvarofelipepupuchemorales/Desktop/Proyecto Economia/assets/PALTA HASS  BCP.xlsx"
hoja_nombre = "Costos Agricolas 01 ha"
celda_objetivo = "E123"
# Conseguimos los precedentes
precedentes_totales = obtener_precedentes_completos(ruta_excel, hoja_nombre, celda_objetivo)
for p in precedentes_totales:
    print(p)

Costos Agricolas 01 ha!$E$45
Costos Agricolas 01 ha!$E$46
Costos Agricolas 01 ha!$E$48
Costos Agricolas 01 ha!$E$53
Costos Agricolas 01 ha!$E$55
Costos Agricolas 01 ha!$H$95
Costos Agricolas 01 ha!E11
Costos Agricolas 01 ha!E120
Costos Agricolas 01 ha!E29
Costos Agricolas 01 ha!E30
Costos Agricolas 01 ha!E31
Costos Agricolas 01 ha!E32
Costos Agricolas 01 ha!E33
Costos Agricolas 01 ha!E34
Costos Agricolas 01 ha!E41
Costos Agricolas 01 ha!E46
Costos Agricolas 01 ha!E50
Costos Agricolas 01 ha!E53
Costos Agricolas 01 ha!E84
Costos Agricolas 01 ha!E85
Costos Agricolas 01 ha!E86
Costos Agricolas 01 ha!E87
Costos Agricolas 01 ha!E88
Costos Agricolas 01 ha!E89
Costos Agricolas 01 ha!F11
Costos Agricolas 01 ha!F120
Costos Agricolas 01 ha!F15
Costos Agricolas 01 ha!F22
Costos Agricolas 01 ha!F23
Costos Agricolas 01 ha!F31
Costos Agricolas 01 ha!F41
Costos Agricolas 01 ha!F84
Costos Agricolas 01 ha!F85
Costos Agricolas 01 ha!F86
Costos Agricolas 01 ha!F87
Costos Agricolas 01 ha!F88
Costos Agricol

In [3]:
print("="*100)

raices = filtrar_valores_raiz(ruta_excel, precedentes_totales)

print("Raíces encontradas:")
print(len(raices))
for celda, valor in raices.items():
    print(f"{celda} → {valor}")

Raíces encontradas:
50
Costos Agricolas 01 ha!$E$45 → 25
Costos Agricolas 01 ha!$E$46 → 0.9
Costos Agricolas 01 ha!$E$53 → 1600
Costos Agricolas 01 ha!$H$95 → 0.07
Costos Agricolas 01 ha!E29 → 100
Costos Agricolas 01 ha!E46 → 0.9
Costos Agricolas 01 ha!E50 → 0.02
Costos Agricolas 01 ha!E53 → 1600
Costos Agricolas 01 ha!F98 → 30800
Costos Agricolas 01 ha!G15 → 350
Costos Agricolas 01 ha!G16 → 200
Costos Agricolas 01 ha!G17 → 1000
Costos Agricolas 01 ha!G19 → 121
Costos Agricolas 01 ha!G23 → 1200
Costos Agricolas 01 ha!G24 → 750
Costos Agricolas 01 ha!G25 → 200
Costos Agricolas 01 ha!G37 → 200
Costos Agricolas 01 ha!G57 → 0
Costos Agricolas 01 ha!G85 → 0
Costos Agricolas 01 ha!G88 → 3800
Costos Agricolas 01 ha!H95 → 0.07
Costos Agricolas 01 ha!J94 → 30800
Costos Agricolas 01 ha!V15 → 427
Costos Agricolas 01 ha!V17 → 1734
Costos Agricolas 01 ha!V19 → 480
Costos Agricolas 01 ha!V23 → 2200
Costos Agricolas 01 ha!V24 → 1200
Costos Agricolas 01 ha!V37 → 300
INSTALACION Y SIEMBRA!B19 → 900
INS

In [4]:
import openpyxl
import re

def generar_precedentes(ruta_excel, hoja_objetivo, celda_objetivo):
    wb = openpyxl.load_workbook(ruta_excel, data_only=False)
    hoja = wb[hoja_objetivo]

    mapa = {}
    formulas = {}
    raices = {}
    visitadas = set()

    def npv(rate, values):
        return sum(v / (1 + rate) ** (i + 1) for i, v in enumerate(values))

    def obtener_formula(celda):
        valor = hoja[celda].value
        if hoja[celda].data_type == 'f':
            return valor
        return None

    def limpiar_formula(formula):
        return formula[1:] if formula.startswith('=') else formula

    def extraer_referencias(formula):
        sin_hoja = re.sub(r"(?:'[^']+'|[A-Za-z0-9_]+)!", "", formula)
        return re.findall(r'\$?[A-Z]{1,3}\$?[0-9]{1,7}', sin_hoja)

    def expandir_rango(rango):
        """Convierte A1:B1 o A1:A5 en una lista de celdas."""
        from openpyxl.utils import column_index_from_string, get_column_letter

        start, end = rango.split(":")
        start_col, start_row = re.match(r'([A-Z]+)([0-9]+)', start).groups()
        end_col, end_row = re.match(r'([A-Z]+)([0-9]+)', end).groups()

        cols = list(range(column_index_from_string(start_col), column_index_from_string(end_col) + 1))
        rows = list(range(int(start_row), int(end_row) + 1))

        resultado = []
        for col in cols:
            for row in rows:
                resultado.append(f"{get_column_letter(col)}{row}")
        return resultado

    def procesar_celda(celda):
        if celda in visitadas:
            return
        visitadas.add(celda)

        formula = obtener_formula(celda)
        if not formula:
            raices[celda] = hoja[celda].value
            return

        formula_limpia = limpiar_formula(formula)
        formulas[celda] = formula_limpia

        # Reemplazar rangos tipo A1:B1
        rangos = re.findall(r'\$?[A-Z]{1,3}\$?[0-9]+:\$?[A-Z]{1,3}\$?[0-9]+', formula_limpia)
        for rango in rangos:
            celdas = expandir_rango(rango.replace('$', ''))
            reemplazo = "[" + ",".join(f"r('{c}')" for c in celdas) + "]"
            formula_limpia = formula_limpia.replace(rango, reemplazo)

        referencias = extraer_referencias(formula_limpia)
        referencias = [ref.replace('$', '') for ref in referencias]

        for ref in referencias:
            procesar_celda(ref)

        formula_py = formula_limpia
        formula_py = re.sub(r"(?:'[^']+'|[A-Za-z0-9_]+)!", "", formula_py)
        for ref in referencias:
            formula_py = re.sub(rf'\b{re.escape(ref)}\b', f"r('{ref}')", formula_py)

        try:
            mapa[celda] = eval(f"lambda r: {formula_py}", {"npv": npv, "sum": sum, "max": max, "min": min, "abs": abs})
        except Exception as e:
            print(f"[ERROR] al evaluar fórmula en {celda}: {formula_py}")
            print(f"       {e}")

    procesar_celda(celda_objetivo)

    return mapa, formulas, raices

In [5]:
mapa, formulas, raices = generar_precedentes(ruta_excel, hoja_nombre, celda_objetivo)

print(mapa)
print(len(mapa))
print(formulas)
print(len(formulas))
print(raices)
print(len(raices))

[ERROR] al evaluar fórmula en J95: +r('J94')*$H$95
       invalid syntax (<string>, line 1)
[ERROR] al evaluar fórmula en H46: +r('H45')*$E$46
       invalid syntax (<string>, line 1)
[ERROR] al evaluar fórmula en H53: +r('H46')*$E$53
       invalid syntax (<string>, line 1)
[ERROR] al evaluar fórmula en H48: +r('H45')*$E$48
       invalid syntax (<string>, line 1)
[ERROR] al evaluar fórmula en H55: +r('H48')*$E$55
       invalid syntax (<string>, line 1)
[ERROR] al evaluar fórmula en K95: +r('K94')*$H$95
       invalid syntax (<string>, line 1)
[ERROR] al evaluar fórmula en I46: +r('I45')*$E$46
       invalid syntax (<string>, line 1)
[ERROR] al evaluar fórmula en I53: +r('I46')*$E$53
       invalid syntax (<string>, line 1)
[ERROR] al evaluar fórmula en I48: +r('I45')*$E$48
       invalid syntax (<string>, line 1)
[ERROR] al evaluar fórmula en I55: +r('I48')*$E$55
       invalid syntax (<string>, line 1)
[ERROR] al evaluar fórmula en L95: +r('L94')*$H$95
       invalid syntax (<strin